# **Drive Mounting**


YOU SHOULD PROVIDE A VALID PATH TO AKEED's FILES

In [1]:
path = "/content/drive/My Drive/Zindi/Competitions/Akeed_Competition/"

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# **IMPORT DATA**

In [3]:
import gc
import math
import datetime
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings("ignore")

In [4]:
%%time
import pandas as pd
import warnings
warnings.simplefilter('ignore')
test_customers=pd.read_csv(path +'test_customers.csv')
test_locations=pd.read_csv(path +'test_locations.csv')
train_customers=pd.read_csv(path +'train_customers.csv')
train_locations=pd.read_csv(path +'train_locations.csv')
vendors=pd.read_csv(path +'vendors.csv')
vendors = vendors.add_prefix('v_')

orders=pd.read_csv(path +'orders.csv')


CPU times: user 621 ms, sys: 85.3 ms, total: 706 ms
Wall time: 11.4 s


# **Create final train & test**

FUNTION TO REDUCE MEMORY USAGE 

In [5]:
def reduce_mem_usage(df, verbose=True):
  numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
  start_mem = df.memory_usage().sum() / 1024**2
  for col in df.columns:
      col_type = df[col].dtypes
      if col_type in numerics:
          c_min = df[col].min()
          c_max = df[col].max()
          if str(col_type)[:3] == 'int':
              if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                  df[col] = df[col].astype(np.int8)
              elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                  df[col] = df[col].astype(np.int16)
              elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                  df[col] = df[col].astype(np.int32)
              elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                  df[col] = df[col].astype(np.int64)
          else:
              if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                  df[col] = df[col].astype(np.float16)
              elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                  df[col] = df[col].astype(np.float32)
              else:
                  df[col] = df[col].astype(np.float64)

  end_mem = df.memory_usage().sum() / 1024**2
  print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
  print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

  return df

In [6]:
def prepare_orders(orders) :
    """
    function that extracts features from orders by doing some aggregations
    return : aggregated features
    """

    loc_rate = orders.groupby(by="LOCATION_NUMBER",as_index=False).agg({'driver_rating' : ['min','mean'], } )
    orders = pd.merge(orders,loc_rate,how='left',on="LOCATION_NUMBER")

    vendor_stats = orders.groupby(by="vendor_id",as_index=False).agg({ 
        'customer_id':['count','nunique'],
        'item_count':['mean','min' ,'std'],
        'grand_total':['mean','min' ,'std'],
        'driver_rating' : ['mean','std'],
        'deliverydistance' : ['std','mean'],
        
        })
    vendor_stats.columns = ["_".join(x) for x in vendor_stats.columns.ravel()]
    
    vendor_stats['nb_reordered'] = (vendor_stats['customer_id_count']-vendor_stats['customer_id_nunique'])
    
    return vendor_stats

In [7]:
vendors['v_vendor_tag_name'].str.split(',').str.len().fillna(0).astype('int').value_counts()

5     25
4     18
7     13
3     11
10     9
6      5
2      5
9      4
8      4
1      3
0      3
Name: v_vendor_tag_name, dtype: int64

In [8]:
def prepare_vendors(vendors,vendor_stats):
      """
      function that extracts features from vendors
      return : vendors with added features 
      """
      #merging stats with vendors
      vendors = pd.merge(vendors,vendor_stats,how="left",right_on="vendor_id_",left_on="v_id")
      
      
      
      
      ############################################   FEATURE ENGINEERING  ON VENDORS ###############################################
      
      vendors['length_of_tags'] = vendors['v_vendor_tag_name'].str.split(',').str.len().fillna(0).astype('int')
      vendors['Multi_option'] = vendors['length_of_tags'].apply(lambda x : 1 if x>=7 else 0)
      vendors['Medium_option'] = vendors['length_of_tags'].apply(lambda x : 1 if (x<=7 and x>=3) else 0)
      vendors['Low_option'] = vendors['length_of_tags'].apply(lambda x : 1 if (x<=3 ) else 0)

      vendors['quality'] = vendors['length_of_tags'] * vendors['v_vendor_rating']
      
      
      
      #################################     Here we will calculate Hours of work    ################
      
      
      
      vendors['v_total_hours_per_week'] = 0
      vendors['v_total_services_per_week'] = 0

      for i , col in enumerate([('v_sunday_from_time2','v_sunday_to_time2'),('v_monday_from_time2','v_monday_to_time2'),('v_tuesday_from_time2','v_tuesday_to_time2'),('v_wednesday_from_time2','v_wednesday_to_time2'),('v_thursday_from_time2','v_thursday_to_time2'),('v_friday_from_time2','v_friday_to_time2'),('v_saturday_from_time2','v_saturday_to_time2')]) :
            vendors[col[0]] = pd.to_datetime(vendors[col[0]], yearfirst=True)
            vendors[col[1]] = pd.to_datetime(vendors[col[1]], yearfirst=True)
            #hours
            vendors[f'time2_diff_of_day_{i}'] = vendors[col[1]] - vendors[col[0]]
            vendors[f'time2_diff_of_day_{i}']  =vendors[f'time2_diff_of_day_{i}'].apply(number_of_hours)
            vendors['v_total_hours_per_week']  = vendors['v_total_hours_per_week'] + vendors[f'time2_diff_of_day_{i}']

            del vendors[f'time2_diff_of_day_{i}']
            #minutes
            vendors[f'time2_diff_minutes_of_day_{i}'] = vendors[col[1]] - vendors[col[0]]
            
            vendors[f'time2_diff_minutes_of_day_{i}']  =vendors[f'time2_diff_minutes_of_day_{i}'].apply(number_of_minutes)
            vendors[f'estimated_number_of_services_on_day_{i}'] = vendors[f'time2_diff_minutes_of_day_{i}'] // vendors['v_prepration_time']
            vendors['v_total_services_per_week'] = vendors['v_total_services_per_week'] + vendors[f'estimated_number_of_services_on_day_{i}']

            del vendors[f'time2_diff_minutes_of_day_{i}']
            del vendors[f'estimated_number_of_services_on_day_{i}']

        

      return vendors

In [9]:
def pre_final_df(train_customers,train_locations,vendors) :
      """
      function that drops duplicated and renames some columns
      """
      train_customers = train_customers.drop_duplicates("akeed_customer_id").reset_index()
      train_locations=train_locations.rename(columns = {"customer_id":"akeed_customer_id"})
      First_Train= train_customers.merge(train_locations,on="akeed_customer_id",how="right")
      First_Train = First_Train.add_prefix('cust_')
       
      pre_Final_df = vendors.assign(key=1).merge(First_Train.assign(key=1), on='key').drop('key',axis=1)

      return pre_Final_df

SOME USEFUL FUNCTIONS : 

In [10]:
def number_of_hours(duration):
    time_s = duration.total_seconds()
    nb_hours = divmod(time_s, 3600)[0]
    return nb_hours

In [11]:
def number_of_minutes(duration):
    time_s = duration.total_seconds()
    nb_minutes = divmod(time_s, 60)[0]
    return nb_minutes

In [12]:
def days(duration):
    time_s = duration.total_seconds()
    nb_days = divmod(time_s, 86400)[0]
    return nb_days

In [13]:
def haversine(lat1, lng1, lat2, lng2 ):
    
    
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    lat_diff = lat2 - lat1
    long_diff = lng2 - lng1
    distance = np.sin(lat_diff * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(long_diff * 0.5) ** 2
    haversine_ = 2 * 6371 * np.arcsin(np.sqrt(distance))
    return haversine_

In [14]:
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295 # Pi/180
    a = 0.5 - np.cos((lat2 - lat1) * p)/2 + np.cos(lat1 * p) * np.cos(lat2 * p) * (1 - np.cos((lon2 - lon1) * p)) / 2
    return 0.6213712 * 12742 * np.arcsin(np.sqrt(a))

In [15]:
def manhattan_distance(lat1, lng1, lat2, lng2):
    
    h1 = haversine(lat1, lng1, lat1, lng2)
    h2 = haversine(lat1, lng1, lat2, lng1)
    return h1 + h2

In [16]:
def bearing(lat1, lng1, lat2, lng2):
    lon_rad = np.radians(lng2 - lng1)
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    
    return np.degrees(np.arctan2(np.sin(lon_rad) * np.cos(lat2), np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(lon_rad)))

In [17]:
def sphere_volume(r):
  pi = 3.1415926535897931
  V= 4.0/3.0*pi* r**3
  return V 

In [18]:
def radian_conv(degree):
    """
    Return radian.
    """
    return  np.radians(degree) 

In [19]:
def calculateDistance2D(x1,y1,x2,y2):  
  """
  return 2D distance
  """
  dist = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)  
  return dist  

In [20]:
def process_df(df):
    """
    the most important function, it extracts many features based on customer_id and location. Here we extract many distance features because that's what helped the most our model. 
    """

     

     ################################ SIMPLE FEATURE ENGINEERING #########################################################
    df['cust_gender'] = df.cust_gender.str.strip()   # remove spaces
    df.replace({'cust_gender': {'male': 'Male', '?????': 'NaN', '': 'NaN'}}, inplace=True)    
    df.replace({'v_OpeningTime2': {'-': 'NaN'}}, inplace=True)
    
    
    
    
    
    
    
    ################################ Distance FEATURE ENGINEERING #########################################################
    
    df['harvesine_dist'] = haversine(df['cust_latitude'], df['cust_longitude'], df['v_latitude'], df['v_longitude']) /1000
    df['manhattan_dist'] = manhattan_distance(df['cust_latitude'], df['cust_longitude'], df['v_latitude'], df['v_longitude']) /1000
    df['other_dist'] = distance(df['cust_latitude'], df['cust_longitude'], df['v_latitude'], df['v_longitude']) /1000
    df['bearing'] = bearing(df['cust_latitude'], df['cust_longitude'], df['v_latitude'], df['v_longitude']) 
    
    #distance ratio
    df['distance_ratio_1'] = df['harvesine_dist'] / df['v_serving_distance']
    df['distance_ratio_2'] = df['manhattan_dist'] / df['v_serving_distance']
    df['distance_ratio_3'] = df['other_dist'] / df['v_serving_distance']

    #distance diffirence
    df['distance_diff_1'] = df['harvesine_dist'] - df['v_serving_distance']
    df['distance_diff_2'] = df['manhattan_dist'] - df['v_serving_distance']
    df['distance_diff_3'] = df['other_dist'] - df['v_serving_distance']

    

    ################################ LATITUDE & LOGITUDE FEATURE ENGINEERING #########################################################
    # 1-Original Latitude & longitude 
    # Centre 
    df['center_latitude'] = (df['cust_latitude'].values + df['v_latitude'].values) / 2
    df['center_longitude'] = (df['cust_longitude'].values + df['v_longitude'].values) / 2
    # Convulution + ration of diffirence 
    df['conv1'] = df['cust_latitude']*df['v_latitude'] # remove abs
    df['conv2'] = df['cust_longitude']*df['v_longitude'] # remove abs
    
    df['conv3'] = df['conv1'] *df['conv2']

    df['SUM_LAT_LONG'] = df['cust_latitude'] + df['v_latitude'] + df['cust_longitude'] + df['v_longitude'] 

    df['relation1'] = (df['cust_latitude']-df['cust_longitude']) / (df['v_latitude']-df['v_longitude']) # remove abs
    
    df['relation2'] = df['SUM_LAT_LONG'] /   df['conv3'] 
    
    df['relation3'] = (df['cust_latitude']+df['cust_longitude']) / (df['v_latitude']+df['v_longitude']) # new 
    
    # 2- Apply radiant to latitude & longitude 
    df['cust_radiant_latitude'] = radian_conv(df['cust_latitude'])
    df['cust_radiant_longitude'] = radian_conv(df['cust_longitude']) 
    
    df['v_radiant_latitude'] = radian_conv(df['v_latitude']) 
    df['v_radiant_longitude'] = radian_conv(df['v_longitude']) 
    
    df['center_radiant_latitude'] = (df['cust_radiant_latitude'].values + df['v_radiant_latitude'].values) / 2
    df['center_radiant_longitude'] = (df['cust_radiant_longitude'].values + df['v_radiant_longitude'].values) / 2

    df['radiant_conv1'] = abs(df['cust_radiant_latitude']*df['v_radiant_latitude'])
    df['radiant_conv2'] = abs(df['cust_radiant_longitude']*df['v_radiant_longitude'])

    #3-Apply cos and sin : Normalization :) 
      ## customer
    df['cust_latitude_cos'] = np.cos(df['cust_radiant_latitude'])
    df['cust_latitude_sin'] = np.sin(df['cust_radiant_latitude'])
    df['cust_longitude_cos'] = np.cos(df['cust_radiant_longitude'])
    df['cust_longitude_sin'] = np.sin(df['cust_radiant_longitude'])

     ##customer transforming to x y z coordinates 
    df['cust_x_coordinate'] = df['cust_longitude_cos'] * df['cust_latitude_cos']
    df['cust_y_coordinate'] = df['cust_longitude_sin'] * df['cust_latitude_cos']
    df['cust_z_coordinate'] = df['cust_latitude_sin']

     ## VENDOR
    df['v_latitude_cos'] = np.cos(df['v_radiant_latitude'])
    df['v_latitude_sin'] = np.sin(df['v_radiant_latitude'])
    df['v_longitude_cos'] = np.cos(df['v_radiant_longitude'])
    df['v_longitude_sin'] = np.sin(df['v_radiant_longitude'])    

     ## VENDOR transforming to x y z coordinates 
    df['v_x_coordinate'] = df['v_longitude_cos'] * df['v_latitude_cos']
    df['v_y_coordinate'] = df['v_longitude_sin'] * df['v_latitude_cos']
    df['v_z_coordinate'] = df['v_latitude_sin'] 




    # OTHER PROPOSED features 
    df['time_consuming'] = (df['distance_diff_1'] / 60) / (df['v_prepration_time'] * 5 )  

    # distance 2D space between vendor customer 
    df['2D_distance'] = calculateDistance2D(df['cust_x_coordinate'],df['cust_y_coordinate'],df['v_x_coordinate'],df['v_y_coordinate'])

    #sphere volume ########### New 
    df['sphere_cust']= sphere_volume(df['2D_distance'])
    df['sphere_v']= sphere_volume(df['v_serving_distance'])

    

    ################################ TIME FEATURE ENGINEERING #########################################################

    df['cust_created_at'] = pd.to_datetime(df['cust_created_at'], yearfirst=True)
    df['cust_updated_at'] = pd.to_datetime(df['cust_updated_at'], yearfirst=True)
    df['v_created_at'] = pd.to_datetime(df['v_created_at'], yearfirst=True)
    df['v_updated_at'] = pd.to_datetime(df['v_updated_at'], yearfirst=True)
    
    df['year_cust_created_at'] = df['cust_created_at'].dt.year
    df['month_cust_created_at'] = df['cust_created_at'].dt.month
    df['doy_cust_created_at'] = df['cust_created_at'].dt.dayofyear
    df['woy_cust_created_at'] = df['cust_created_at'].dt.weekofyear
    df['wday_cust_created_at'] = df['cust_created_at'].dt.weekday

    df['year_cust_updated_at'] = df['cust_updated_at'].dt.year
    df['month_cust_updated_at'] = df['cust_updated_at'].dt.month
    df['doy_cust_updated_at'] = df['cust_updated_at'].dt.dayofyear
    df['woy_cust_updated_at'] = df['cust_updated_at'].dt.weekofyear
    df['wday_cust_updated_at'] = df['cust_updated_at'].dt.weekday

    df['year_v_created_at'] = df['v_created_at'].dt.year
    df['month_v_created_at'] = df['v_created_at'].dt.month
    df['doy_v_created_at'] = df['v_created_at'].dt.dayofyear
    df['woy_v_created_at'] = df['v_created_at'].dt.weekofyear
    df['wday_v_created_at'] = df['v_created_at'].dt.weekday

    df['year_v_updated_at'] = df['v_updated_at'].dt.year
    df['month_v_updated_at'] = df['v_updated_at'].dt.month
    df['doy_v_updated_at'] = df['v_updated_at'].dt.dayofyear
    df['woy_v_updated_at'] = df['v_updated_at'].dt.weekofyear
    df['wday_v_updated_at'] = df['v_updated_at'].dt.weekday
    
    df['cust_diff_update_create'] = df['cust_updated_at'] - df['cust_created_at']
    df['v_diff_update_create'] = df['v_updated_at'] - df['v_created_at']
    df['cust_v_diff_create'] = df['v_created_at'] - df['cust_created_at']
    df['cust_v_diff_update'] = df['v_updated_at'] - df['cust_updated_at']

    df['cust_diff_update_create'] = df['cust_diff_update_create'].apply(days)
    df['v_diff_update_create'] = df['v_diff_update_create'].apply(days)
    df['cust_v_diff_create'] = df['cust_v_diff_create'].apply(days)
    df['cust_v_diff_update'] = df['cust_v_diff_update'].apply(days)
    
    
    del df['cust_created_at']
    del df['cust_updated_at']
    del df['v_created_at']
    del df['v_updated_at']
    gc.collect()
    
    
    #################################   Statistics Based on CUSTOMER ID :D ################################################################
    distance_statique = df.groupby(by="cust_akeed_customer_id",as_index=False).agg({ 
        'distance_diff_1':['mean','std'],
        'distance_diff_2':['mean','std'],
        'distance_diff_3':['std','mean'],
        
        'distance_ratio_1' : ['std','mean'] ,
        'distance_ratio_2' : ['std','mean'],
        
        'harvesine_dist' : ['std','mean'],
        'bearing' : ['std','mean','median','min','max'], ## add median min max 
        
        'relation1' :['std','mean'],
        'center_longitude' : ['std','mean'] ,
        'center_latitude' : ['std','mean'] ,

        
        'cust_diff_update_create' : ['mean','std','max','min'],
        'cust_v_diff_create' : ['mean','std','max','min'],
        
        
        'conv1' : ['mean','std'],
        'conv2' : ['mean','std'],
        'conv3' : ['std','mean'], 
        

        'radiant_conv1' : ['mean','std'],
        'radiant_conv2' : ['mean','std'],
        'time_consuming' : ['std','mean'],
        
        '2D_distance' : ['std','mean','max','min','median'],
        'sphere_cust' : ['std','mean','max'], 
        'sphere_v' : ['std','mean','max'] 




        })
    distance_statique.columns = ["_".join(x) for x in distance_statique.columns.ravel()]
    df = pd.merge(df,distance_statique,how="left",right_on="cust_akeed_customer_id_",left_on="cust_akeed_customer_id")





    
    return df

# **APPLICATION OF CREATED FUNCTIONS**

In [21]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
def Label_enc(df) :
    """
    function that label encodes a set of columnes
    
    """
    le_cols =  ['v_sunday_from_time1','v_sunday_to_time1','v_monday_from_time1','v_monday_to_time1','v_tuesday_from_time1','v_tuesday_to_time1',
                 'v_wednesday_from_time1','v_wednesday_to_time1','v_thursday_from_time1','v_thursday_to_time1','v_friday_from_time1',
                 'v_friday_to_time1','v_saturday_from_time1','v_saturday_to_time1',
                 'v_sunday_from_time2','v_sunday_to_time2','v_monday_from_time2','v_monday_to_time2','v_tuesday_from_time2','v_tuesday_to_time2',
                 'v_wednesday_from_time2','v_wednesday_to_time2','v_thursday_from_time2','v_thursday_to_time2','v_friday_from_time2',
                 'v_friday_to_time2','v_saturday_from_time2','v_saturday_to_time2',]
    for col in le_cols:
    
      df[col][pd.isnull(df[col])]  = 'nan'
      df[col+'LAB_ENC']= le.fit_transform(df[col])
    return df

In [22]:
%%time
vendors = Label_enc(vendors)

CPU times: user 58.6 ms, sys: 1.45 ms, total: 60.1 ms
Wall time: 58.8 ms


In [23]:
%%time
vendor_stats = prepare_orders(orders)
vendors = prepare_vendors(vendors,vendor_stats)

CPU times: user 392 ms, sys: 32.9 ms, total: 425 ms
Wall time: 482 ms


**DROPPING STATIC, CORRELATED AND NON IMPORTANT COLUMNS**

In [24]:
vendors.drop(['v_sunday_from_time1','v_sunday_to_time1','v_monday_from_time1','v_monday_to_time1','v_tuesday_from_time1','v_tuesday_to_time1',
                 'v_wednesday_from_time1','v_wednesday_to_time1','v_thursday_from_time1','v_thursday_to_time1','v_friday_from_time1',
                 'v_friday_to_time1','v_saturday_from_time1','v_saturday_to_time1',
                 'v_sunday_from_time2','v_sunday_to_time2','v_monday_from_time2','v_monday_to_time2','v_tuesday_from_time2','v_tuesday_to_time2',
                 'v_wednesday_from_time2','v_wednesday_to_time2','v_thursday_from_time2','v_thursday_to_time2','v_friday_from_time2',
                 'v_friday_to_time2','v_saturday_from_time2','v_saturday_to_time2',
              'v_monday_from_time1LAB_ENC',	'v_monday_to_time1LAB_ENC'	,'v_tuesday_from_time1LAB_ENC'	,'v_tuesday_to_time1LAB_ENC',	'v_wednesday_from_time1LAB_ENC',
'v_thursday_from_time1LAB_ENC'	,'v_thursday_to_time1LAB_ENC',	'v_friday_from_time1LAB_ENC',	'v_friday_to_time1LAB_ENC' ,'v_wednesday_to_time1LAB_ENC',
'v_friday_from_time2LAB_ENC',	'v_friday_to_time2LAB_ENC', 'vendor_id_','customer_id_count','customer_id_nunique',''
'v_monday_from_time2LAB_ENC' ,	'v_monday_to_time2LAB_ENC',	'v_tuesday_from_time2LAB_ENC',	'v_tuesday_to_time2LAB_ENC',
'v_wednesday_from_time2LAB_ENC',	'v_wednesday_to_time2LAB_ENC'	,'v_thursday_from_time2LAB_ENC',	'v_thursday_to_time2LAB_ENC',],1,inplace=True)

In [25]:
%%time
pre_Final_train = pre_final_df(train_customers, train_locations, vendors)
pre_Final_test = pre_final_df(test_customers, test_locations, vendors)

CPU times: user 16.3 s, sys: 2.08 s, total: 18.4 s
Wall time: 18.4 s


In [26]:
%%time

pre_Final_train = reduce_mem_usage(pre_Final_train)
pre_Final_test = reduce_mem_usage(pre_Final_test)

Memory usage after optimization is: 1327.87 MB
Decreased by 58.8%
Memory usage after optimization is: 373.12 MB
Decreased by 58.8%
CPU times: user 12 s, sys: 59.8 ms, total: 12 s
Wall time: 12 s


In [27]:
%%time

train = process_df(pre_Final_train)
test = process_df(pre_Final_test)

CPU times: user 5min 6s, sys: 2.79 s, total: 5min 9s
Wall time: 5min 9s


In [28]:
%%time
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

Memory usage after optimization is: 2888.40 MB
Decreased by 26.9%
Memory usage after optimization is: 811.62 MB
Decreased by 26.9%
CPU times: user 24.1 s, sys: 221 ms, total: 24.3 s
Wall time: 24.3 s


In [29]:
del pre_Final_train
del pre_Final_test
gc.collect()

0

In [30]:
le = LabelEncoder()
def Label_enc(df) :
    le_cols =  ['cust_location_type','v_OpeningTime','v_OpeningTime2',
                  'v_vendor_category_en',
                  'cust_gender',] 
                 
                
    le_cols = list(set(le_cols))
    for col in le_cols:
      print(30 * '=')
      print('Label_enc TO ---> : ' ,col)
      df[col][pd.isnull(df[col])]  = 'nan'
      df[col]= le.fit_transform(df[col])
    return df

In [31]:
%%time
train = Label_enc(train)
test = Label_enc(test)

Label_enc TO ---> :  cust_location_type
Label_enc TO ---> :  v_OpeningTime2
Label_enc TO ---> :  v_OpeningTime
Label_enc TO ---> :  cust_gender
Label_enc TO ---> :  v_vendor_category_en
Label_enc TO ---> :  cust_location_type
Label_enc TO ---> :  v_OpeningTime2
Label_enc TO ---> :  v_OpeningTime
Label_enc TO ---> :  cust_gender
Label_enc TO ---> :  v_vendor_category_en
CPU times: user 11 s, sys: 346 ms, total: 11.3 s
Wall time: 11.3 s


In [32]:
orders=pd.read_csv(path +'orders.csv')

def final_train(train,orders):
    train["CID X LOC_NUM X VENDOR"] = train["cust_akeed_customer_id"].astype(str)+' X '+ train["cust_location_number"].astype(str)+' X '+ train["v_id"].astype(str)

    all_services = orders['CID X LOC_NUM X VENDOR'].values.tolist()
    train['target']= train['CID X LOC_NUM X VENDOR'].isin(all_services).astype(int).values
    return train

def final_test(test) :
    test["CID X LOC_NUM X VENDOR"] = test["cust_akeed_customer_id"].astype(str)+' X '+ test["cust_location_number"].astype(str)+' X '+ test["v_id"].astype(str)
    return test

In [33]:
%%time
train = final_train(train,orders)
test = final_test(test)

CPU times: user 10.8 s, sys: 1.07 s, total: 11.9 s
Wall time: 11.9 s


In [34]:
train.drop(['cust_index'] ,1,inplace=True)
test.drop(['cust_index'] ,1,inplace=True)

In [35]:
%%time
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

Memory usage after optimization is: 2729.51 MB
Decreased by 8.0%
Memory usage after optimization is: 765.38 MB
Decreased by 6.8%
CPU times: user 21.7 s, sys: 59.6 ms, total: 21.8 s
Wall time: 21.8 s


In [36]:
train.shape ,test.shape

((5950300, 191), (1672000, 190))

# **MODELING LGBM**

In [37]:
import pandas as pd
import numpy as np
import math
from numba import jit
import lightgbm as lgb
import time

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import f1_score, classification_report
from scipy.misc import derivative

import matplotlib.pyplot as plt
import seaborn as sns

import gc
import warnings
warnings.filterwarnings("ignore")

**CHECK STATIC COLUMNS (WITH ONLY ONE VALUE)**

In [38]:
for  col in list(train.columns) :
  if len(list(train[col].unique())) == 1 :
    print(str(col) )

v_is_akeed_delivering
v_open_close_flags
v_one_click_vendor
v_country_id
v_city_id
v_display_orders
item_count_min
year_v_updated_at
sphere_v_std
sphere_v_mean
sphere_v_max


**DROPPING NON IMPORTANT COLUMNS FROM FINAL DATA**

In [39]:
seed = 849
np.random.seed(seed)



to_drop = ['cust_akeed_customer_id_',
           'v_vendor_rating',	'v_primary_tags',	'v_vendor_tag_name',
           'v_monday_from_time1LAB_ENC',	'v_monday_to_time1LAB_ENC'	,'v_tuesday_from_time1LAB_ENC'	,'v_tuesday_to_time1LAB_ENC',	'v_wednesday_from_time1LAB_ENC',
'v_thursday_from_time1LAB_ENC'	,'v_thursday_to_time1LAB_ENC',	'v_friday_from_time1LAB_ENC',	'v_friday_to_time1LAB_ENC' ,'v_wednesday_to_time1LAB_ENC',
'v_friday_from_time2LAB_ENC',	'v_friday_to_time2LAB_ENC', 'vendor_id_','customer_id_count','customer_id_nunique',''
'v_monday_from_time2LAB_ENC' ,	'v_monday_to_time2LAB_ENC',	'v_tuesday_from_time2LAB_ENC',	'v_tuesday_to_time2LAB_ENC',
'v_wednesday_from_time2LAB_ENC',	'v_wednesday_to_time2LAB_ENC'	,'v_thursday_from_time2LAB_ENC',	'v_thursday_to_time2LAB_ENC',

           'cust_akeed_customer_id','v_is_akeed_delivering', 'v_open_close_flags','v_one_click_vendor', 'v_country_id','v_city_id',
           'v_display_orders','cust_customer_id','CID X LOC_NUM X VENDOR','v_authentication_id','cust_language','v_language',
           'v_vendor_tag', 'target',
           'v_open_close_flags',
'v_country_id',
'v_city_id',
'v_display_orders',
'item_count_min',
'nb_reordered',
'v_nb_tags',
'v_is_akeed_delivering',
'v_open_close_flags',
'v_one_click_vendor',
'v_country_id',
'v_city_id',
'v_display_orders',
'item_count_min',
'year_v_updated_at',]

features = [c for c in train.columns if c not in to_drop]
len(features)

171

**HERE WE DO OUR SPLIT ( Based on New customers )**

In [40]:
%%time
train = train.sort_values(by='cust_akeed_customer_id', ascending=False).reset_index(drop=True)


target = train['target']
X_test = train[features][:892500].values
y_test = target[:892500].values

X_train = train[features][892500:].values
y_train = target[892500:].values

trn_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_test, label=y_test)

CPU times: user 20.6 s, sys: 1.05 s, total: 21.6 s
Wall time: 21.6 s


**Training Takes UP to 2h to run**

In [ ]:
%%time

weight = 5870158/80142/10
param = {
    'bagging_freq': 1,
    'bagging_fraction': 0.8,
    'feature_fraction': 0.9,
    'boost_from_average':'false',
    'num_leaves': 90,
    'boost': 'gbdt',
    'learning_rate': 0.02,
    'metric':'auc',
    'scale_pos_weight': round(weight, 1),
    'tree_learner': 'serial',
    'objective': 'binary',
    'random_state': seed,
    'n_jobs': -1,
    'verbosity': -1
}

lgb_model = lgb.train(param, trn_data, 5000, valid_sets = [trn_data, val_data], verbose_eval=200, early_stopping_rounds=200)

Training until validation scores don't improve for 200 rounds.
[200]	training's auc: 0.918955	valid_1's auc: 0.917123
[400]	training's auc: 0.929978	valid_1's auc: 0.922986
[600]	training's auc: 0.936566	valid_1's auc: 0.924705
[800]	training's auc: 0.941516	valid_1's auc: 0.925494
[1000]	training's auc: 0.945757	valid_1's auc: 0.925983
[1200]	training's auc: 0.9494	valid_1's auc: 0.926372


**VALIDATION AND PREDICTION :**

In [ ]:
pred = lgb_model.predict(X_test)

**HERE WE TRY TO FIND THE BEST THRESHOLD ON VALIDATION SET**

In [ ]:
best_score = -1
best_threshold = 0
score= 0 
for i in np.arange(0.45, 0.55, 0.0005):

  pred_lgb = (pred >= i)*1
  score = f1_score(y_test, pred_lgb) 
  if score > best_score:
    best_score= score
    best_threshold = i
    print("Current Best threshold : ",best_threshold )
    print("Current Best Score : ",best_score )
    print("********************")

In [ ]:
pred = (pred >= best_threshold)*1
print('Best F1 Score : ', f1_score(y_test, pred))

In [ ]:
print(classification_report(y_test, pred))

In [ ]:
test_id = test['CID X LOC_NUM X VENDOR']
full_pred = lgb_model.predict(test[features].values)

In [ ]:
submission = pd.DataFrame()
submission['CID X LOC_NUM X VENDOR'] = test_id
submission['target'] = (full_pred >= best_threshold)*1
submission.target.value_counts()

In [ ]:
submission.target.mean()

In [ ]:
submission.to_csv("AKEED_LGBM_NACIR_BINARY.csv",index=False)

# **FLOATS PREDICTIONS : ( This part is needed for blend)**

In [ ]:
submission = pd.DataFrame()
submission['CID X LOC_NUM X VENDOR'] = test_id
submission['target'] = full_pred

In [ ]:
submission.head()

In [ ]:
submission.to_csv("AKEED_LGBM_NACIR_FLOATS.csv",index=False)